In [2]:
import os
import openai
import mlflow
client = openai.OpenAI(
    api_key=os.getenv('OPENAI_API_KEY'),
    base_url=os.getenv('OPENAI_BASE_URL')
)
mlflow.set_tracking_uri("http://127.0.0.1:5001")

In [ ]:
import mlflow
import openai
import subprocess


# Configure MLflow Tracking
mlflow.set_experiment("my-genai-app")

# Get current git commit hash
git_commit = (
    subprocess.check_output(["git", "rev-parse", "HEAD"]).decode("ascii").strip()[:8]
)

# Define your application version using the git commit
app_name = "customer_support_agent"
logged_model_name = f"{app_name}-{git_commit}"

# Set the active model context - traces will be linked to this
mlflow.set_active_model(name=logged_model_name)

# Enable autologging for OpenAI, which automatically logs traces
mlflow.openai.autolog()

# Define and test your agent code - traces are automatically linked
client = openai.OpenAI()
questions = [
    "How do I reset my password?",
    "What are your business hours?",
    "Can I get a refund for my last order?",
    "Where can I find the user manual for product model number 15869?",
    "I'm having trouble with the payment page, can you help?",
]
for question in questions:
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": question}],
        temperature=0.7,
        max_tokens=1000,
    )